# CME 257 - Julia Packages

(This IJulia notebook is tested using Julia [v0.4.0-rc3](https://github.com/JuliaLang/julia/releases))

Julia currently has [over 700 packages](http://pkg.julialang.org/pulse.html) registered.  This includes relatively mature and popular pacakges as well as packages that are just getting off the ground.

Today we'll look a bit more at the package manager and some popular Julia packages.

You have already used [Pkg.add()](http://julia.readthedocs.org/en/latest/stdlib/pkg/#Base.Pkg.add) to start using packages such as IJulia and PyPlot.  You can also add a version number to the arguments to use a particular version of a package.  You can use [Pkg.available()](http://julia.readthedocs.org/en/latest/stdlib/pkg/#Base.Pkg.available) to list available versions of a package.

In [ ]:
Pkg.available("Gadfly")

Recall that you can use Pkg.status() to view all your installed packages.  Required pacakges are those that you specifically added.  Additional packages are those that are required as dependencies of the required packages.

In [ ]:
Pkg.status()

If you want to see if a particular package is installed, you can use Pkg.installed()

In [ ]:
Pkg.installed("IJulia")

Sometimes after updating a package you may need to re-build it.

In [ ]:
Pkg.build("Gadfly")

If you're having trouble getting packages to play nicely, you can try using [Pkg.resolve()](http://julia.readthedocs.org/en/latest/stdlib/pkg/#Base.Pkg.resolve).

In [ ]:
Pkg.resolve()

If you're debugging a package, Pkg.test() will run tests provided with a package

In [ ]:
Pkg.test("PyCall")

If you want to use a package that is not a registered Julia package or if you want to use the current development branch of a package, use Pkg.clone() with a GitHub URL.

In [ ]:
Pkg.clone("git@github.com:JuliaSparse/Metis.jl.git")

In [ ]:
Pkg.status()

In [ ]:
Pkg.rm("Metis")

In [ ]:
Pkg.status()

# Interact.jl

Julia's [interact package](https://github.com/JuliaLang/Interact.jl) lets you create widgets that you can manipulate (similar to Mathematica)

In [ ]:
using Interact
using Gadfly
# Interact plays nicely with Gadfly
# you may have trouble with PyPlot.

In [ ]:
# this example is from the Interact.jl GitHub page: https://github.com/JuliaLang/Interact.jl
@manipulate for ϕ=0:π/16:4π, f=[:sin => sin, :cos => cos]
    plot(θ -> f(θ + ϕ), 0, 25)
end

In [ ]:
using AudioIO, Interact, Gadfly

In [ ]:
# example due to Sashi Gowda https://github.com/shashi
# source: https://github.com/JuliaLang/Interact.jl/issues/36
s1 = SinOsc(220)
s2 = SinOsc(220)
@manipulate for f1=100:880, f2 = 110:880
    s1.renderer.freq = f1
    s2.renderer.freq = f2
    plot(t->sin(f1*2pi*t) + sin(f2*2pi*t), 0, 2pi)
end
play(s1)
play(s2)
;

In [ ]:
stop(s1)
stop(s2)

In [ ]:
using Colors, Compose, Interact

In [ ]:
# example due to Steven Johnson https://github.com/stevengj
# source: https://github.com/JuliaLang/Interact.jl/issues/36
const colors = distinguishable_colors(6)
function sierpinski(n, colorindex=1)
    if n == 0
        compose(context(), circle(0.5,0.5,0.5), fill(colors[colorindex]))
    else
        colorindex = colorindex % length(colors) + 1
        t1 = sierpinski(n - 1, colorindex)
        colorindex = colorindex % length(colors) + 1
        t2 = sierpinski(n - 1, colorindex)
        colorindex = colorindex % length(colors) + 1
        t3 = sierpinski(n - 1, colorindex)
        compose(context(),
                (context(1/4,   0, 1/2, 1/2), t1),
                (context(  0, 1/2, 1/2, 1/2), t2),
                (context(1/2, 1/2, 1/2, 1/2), t3))
    end
end

@manipulate for n = 1:8
    sierpinski(n)
end

# DataFrames

Julia has a [DataFrames](https://github.com/JuliaStats/DataFrames.jl) package to help you work with data sets and tabular data.  You can also add the [RDatasets package](https://github.com/johnmyleswhite/RDatasets.jl) to access classical datasets to play around with.

In [ ]:
using RDatasets
iris = dataset("datasets", "iris")
@show typeof(iris)
;

You can use DataFrames with Gadfly for R-like plotting

In [ ]:
# example from Gadfly home: https://github.com/dcjones/Gadfly.jl
using Gadfly
plot(iris, x="SepalLength", y="SepalWidth", Geom.point)

# Exercise 1
* Add Gadfly, DataFrames, and RDatasets to your packages.
* load the "iris" dataset from the "dataset" package in RDatasets (or the dataset of your choice)
* play around with Gadfly or other plotting package, and see if you can find a cool way to visualize the data (don't just use the above example)

# Distributions

Sometimes you may want to draw random numbers from a distribution other than the normal (Julia's randn()), and uniform (Julia's rand()) distributions.  The [Distributions package](https://github.com/JuliaStats/Distributions.jl) provides these distributions for you.

In [ ]:
using Distributions

In [ ]:
srand(1000) # set random seed
normal_d = Normal() # create distribution

In [ ]:
x = rand(normal_d, 100) # create an array of 100 random entries drawn from the distribution

# Exercise 2

* combine the distributions package and interact package to visualize the pdf of an exponential distribution for different values of $\beta$.
* modify the code to visualize a different distribution of your choice.

see [univariate distributions](http://distributionsjl.readthedocs.org/en/latest/univariate.html)

# Optimization Packages

Julia's optimization packages are some of its most popular features.  You can start exploring on [http://www.juliaopt.org/](http://www.juliaopt.org/).  

If you're not familiar with numerical optimization, you may find the JuliaOpt website a bit confusing at first, so here's a basic primer:

Numerical optimization seeks to find solutions so problems that are generally of the form

$$ \min_x f(x) \qquad s.t.~c(x) \le b $$

For example, you may wish to minimize 

$$ f(x) = x^2 $$

with no constraints.  The idea is that you want to give this function to a computer and it will tell you the answer.  Yes, you can minimize this particular example by hand, but you may also wish to optimize a function that is not as trivial.

Typically, there are several layers to numerical optimization, which we can see in JuliaOpt.  The top level, which users usually interact with are algebraic modelling languages which take human-readable input and transform it into a problem that the computer can solve.  JuliaOpt's algebraic modelling languages are:
* JuMP (produced by Miles Lubin and Iain Dunning at MIT)
* Convex (similar to CVX - it also comes out of Stephen Boyd's group)

Your choice of modelling language will depend on what kind of problem you want to solve, and what you are familiar with.

The layer beneath the modelling language is MathProgBase, which standardizes interfaces to the solvers.  Solvers are at the bottom level, and they implement specific routines for particular types of optimization problems.  These do the heavy lifting for computations.

Julia also has some stand-alone packages that don't fall under this heirarchy:
* Optim
* LsqFit

Here are two examples that use optimization packages:

In [ ]:
using Optim

In [ ]:
f(x::Vector) = x.^2

optimize(f, -1.0, 1.0)

In [ ]:
using Convex, SCS

In [ ]:
# from http://convexjl.readthedocs.org/en/latest/quick_tutorial.html
m = 4;  n = 5
A = randn(m, n); b = randn(m, 1)

# Create a (column vector) variable of size n x 1.
x = Variable(n)

# The problem is to minimize ||Ax - b||^2 subject to x >= 0
# This can be done by: minimize(objective, constraints)
problem = minimize(sum_squares(A * x - b), [x >= 0])

# Solve the problem by calling solve!
solve!(problem)

# Check the status of the problem
problem.status # :Optimal, :Infeasible, :Unbounded etc.

# Get the optimum value
problem.optval

@show x.value
;

# Exercise 3

* Use an optimization package to minimize the following function
$$F(x,y) = (x-1)^2 + (y-2)^2$$